# Resampling Wheel

Now that you’ve learned the resampling wheel pseudo code, you'll try to implement it in C++. In this quiz, **resample** the particles with a sample probability proportional to the importance weight.

![](images/resampling-wheel.png)
![](images/step-1.png)
![](images/step-2.png)
![](images/step-3.png)

![](images/09-resampling-wheel.00-02-05-24.still002.png)

In [1]:
//#include "src/matplotlibcpp.h"//Graph Library
#include <iostream>
#include <string>
#include <math.h>
#include <vector>
#include <stdexcept> // throw errors
#include <random> //C++ 11 Random Numbers

In [2]:
//namespace plt = matplotlibcpp;
using namespace std;

In [3]:
// Landmarks
double landmarks[8][2] = { { 20.0, 20.0 }, { 20.0, 80.0 }, { 20.0, 50.0 },
    { 50.0, 20.0 }, { 50.0, 80.0 }, { 80.0, 80.0 },
    { 80.0, 20.0 }, { 80.0, 50.0 } };

// Map size in meters
double world_size = 100.0;

// Random Generators
std::random_device rd;
std::mt19937 gen(rd());

In [4]:
double gen_real_random()
{
    // Generate real random between 0 and 1
    uniform_real_distribution<double> real_dist(0.0, 1.0); //Real
    return real_dist(gen);
}

In [5]:
double mod(double first_term, double second_term)
{
    // Compute the modulus
    return first_term - (second_term)*floor(first_term / (second_term));
}

In [6]:
class Robot {
public:
    Robot()
    {
        // Constructor
        x = gen_real_random() * world_size; // robot's x coordinate
        y = gen_real_random() * world_size; // robot's y coordinate
        orient = gen_real_random() * 2.0 * M_PI; // robot's orientation

        forward_noise = 0.0; //noise of the forward movement
        turn_noise = 0.0; //noise of the turn
        sense_noise = 0.0; //noise of the sensing
    }

    void set(double new_x, double new_y, double new_orient)
    {
        // Set robot new position and orientation
        if (new_x < 0 || new_x >= world_size)
            throw std::invalid_argument("X coordinate out of bound");
        if (new_y < 0 || new_y >= world_size)
            throw std::invalid_argument("Y coordinate out of bound");
        if (new_orient < 0 || new_orient >= 2 * M_PI)
            throw std::invalid_argument("Orientation must be in [0..2pi]");

        x = new_x;
        y = new_y;
        orient = new_orient;
    }

    void set_noise(double new_forward_noise, double new_turn_noise, double new_sense_noise)
    {
        // Simulate noise, often useful in particle filters
        forward_noise = new_forward_noise;
        turn_noise = new_turn_noise;
        sense_noise = new_sense_noise;
    }

    vector<double> sense()
    {
        // Measure the distances from the robot toward the landmarks
        vector<double> z(sizeof(landmarks) / sizeof(landmarks[0]));
        double dist;

        for (int i = 0; i < sizeof(landmarks) / sizeof(landmarks[0]); i++) {
            dist = sqrt(pow((x - landmarks[i][0]), 2) + pow((y - landmarks[i][1]), 2));
            dist += gen_gauss_random(0.0, sense_noise);
            z[i] = dist;
        }
        return z;
    }

    Robot move(double turn, double forward)
    {
        if (forward < 0)
            throw std::invalid_argument("Robot cannot move backward");

        // turn, and add randomness to the turning command
        orient = orient + turn + gen_gauss_random(0.0, turn_noise);
        orient = mod(orient, 2 * M_PI);

        // move, and add randomness to the motion command
        double dist = forward + gen_gauss_random(0.0, forward_noise);
        x = x + (cos(orient) * dist);
        y = y + (sin(orient) * dist);

        // cyclic truncate
        x = mod(x, world_size);
        y = mod(y, world_size);

        // set particle
        Robot res;
        res.set(x, y, orient);
        res.set_noise(forward_noise, turn_noise, sense_noise);

        return res;
    }

    string show_pose()
    {
        // Returns the robot current position and orientation in a string format
        return "[x=" + to_string(x) + " y=" + to_string(y) + " orient=" + to_string(orient) + "]";
    }

    string read_sensors()
    {
        // Returns all the distances from the robot toward the landmarks
        vector<double> z = sense();
        string readings = "[";
        for (int i = 0; i < z.size(); i++) {
            readings += to_string(z[i]) + " ";
        }
        readings[readings.size() - 1] = ']';

        return readings;
    }

    double measurement_prob(vector<double> measurement)
    {
        // Calculates how likely a measurement should be
        double prob = 1.0;
        double dist;

        for (int i = 0; i < sizeof(landmarks) / sizeof(landmarks[0]); i++) {
            dist = sqrt(pow((x - landmarks[i][0]), 2) + pow((y - landmarks[i][1]), 2));
            prob *= gaussian(dist, sense_noise, measurement[i]);
        }

        return prob;
    }

    double x, y, orient; //robot poses
    double forward_noise, turn_noise, sense_noise; //robot noises

private:
    double gen_gauss_random(double mean, double variance)
    {
        // Gaussian random
        normal_distribution<double> gauss_dist(mean, variance);
        return gauss_dist(gen);
    }

    double gaussian(double mu, double sigma, double x)
    {
        // Probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(-(pow((mu - x), 2)) / (pow(sigma, 2)) / 2.0) / sqrt(2.0 * M_PI * (pow(sigma, 2)));
    }
};

In [7]:
double evaluation(Robot r, Robot p[], int n)
{
    //Calculate the mean error of the system
    double sum = 0.0;
    for (int i = 0; i < n; i++) {
        //the second part is because of world's cyclicity
        double dx = mod((p[i].x - r.x + (world_size / 2.0)), world_size) - (world_size / 2.0);
        double dy = mod((p[i].y - r.y + (world_size / 2.0)), world_size) - (world_size / 2.0);
        double err = sqrt(pow(dx, 2) + pow(dy, 2));
        sum += err;
    }
    return sum / n;
}

In [8]:
double max(double arr[], int n)
{
    // Identify the max element in an array
    double max = 0;
    for (int i = 0; i < n; i++) {
        if (arr[i] > max)
            max = arr[i];
    }
    return max;
}

In [9]:
void run()
{
    Robot myrobot;
    myrobot.set_noise(5.0, 0.1, 5.0);
    myrobot.set(30.0, 50.0, M_PI / 2.0);
    myrobot.move(-M_PI / 2.0, 15.0);
    //cout << myrobot.read_sensors() << endl;
    myrobot.move(-M_PI / 2.0, 10.0);
    //cout << myrobot.read_sensors() << endl;

    // Create a set of particles
    int n = 1000;
    Robot p[n];

    for (int i = 0; i < n; i++) {
        p[i].set_noise(0.05, 0.05, 5.0);
        //cout << p[i].show_pose() << endl;
    }

    //Re-initialize myrobot object and Initialize a measurment vector
    myrobot = Robot();
    vector<double> z;

    //Move the robot and sense the environment afterwards
    myrobot = myrobot.move(0.1, 5.0);
    z = myrobot.sense();

    // Simulate a robot motion for each of these particles
    Robot p2[n];
    for (int i = 0; i < n; i++) {
        p2[i] = p[i].move(0.1, 5.0);
        p[i] = p2[i];
    }

    //Generate particle weights depending on robot's measurement
    double w[n];
    for (int i = 0; i < n; i++) {
        w[i] = p[i].measurement_prob(z);
        //cout << w[i] << endl;
    }

    //####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####

    //Resample the particles with a sample probability proportional to the importance weight
    Robot p3[n];
    int index = gen_real_random() * n;
    //cout << index << endl;
    double beta = 0.0;
    double mw = max(w, n);
    //cout << mw;
    for (int i = 0; i < n; i++) {
        beta += gen_real_random() * 2.0 * mw;
        while (beta > w[index]) {
            beta -= w[index];
            index = mod((index + 1), n);
        }
        p3[i] = p[index];
    }
    for (int k=0; k < n; k++) {
        p[k] = p3[k];
        cout << p[k].show_pose() << endl;
    }

    return 0;
}

In [10]:
run()

[x=41.695354 y=16.562889 orient=2.126091]
[x=48.245957 y=10.061466 orient=4.611104]
[x=38.589867 y=11.880464 orient=3.575306]
[x=45.082584 y=9.309207 orient=2.759685]
[x=45.870849 y=12.716931 orient=0.496268]
[x=44.468963 y=17.342648 orient=0.340112]
[x=41.564902 y=11.361145 orient=5.771038]
[x=41.695354 y=16.562889 orient=2.126091]
[x=45.870849 y=12.716931 orient=0.496268]
[x=43.491410 y=12.389155 orient=2.491543]
[x=46.452010 y=16.655128 orient=2.195869]
[x=41.695354 y=16.562889 orient=2.126091]
[x=45.870849 y=12.716931 orient=0.496268]
[x=46.939002 y=14.057148 orient=0.393008]
[x=37.348039 y=13.952389 orient=3.678241]
[x=41.564902 y=11.361145 orient=5.771038]
[x=46.452010 y=16.655128 orient=2.195869]
[x=41.695354 y=16.562889 orient=2.126091]
[x=44.468963 y=17.342648 orient=0.340112]
[x=37.348039 y=13.952389 orient=3.678241]
[x=46.452010 y=16.655128 orient=2.195869]
[x=46.452010 y=16.655128 orient=2.195869]
[x=46.452010 y=16.655128 orient=2.195869]
[x=47.451792 y=14.176535 orient=0.4

[x=44.468963 y=17.342648 orient=0.340112]
[x=37.348039 y=13.952389 orient=3.678241]
[x=44.660626 y=20.822293 orient=4.887577]
[x=41.695354 y=16.562889 orient=2.126091]
[x=46.939002 y=14.057148 orient=0.393008]
[x=38.589867 y=11.880464 orient=3.575306]
[x=38.885777 y=12.888064 orient=5.045362]
[x=47.451792 y=14.176535 orient=0.497455]
[x=41.695354 y=16.562889 orient=2.126091]
[x=45.870849 y=12.716931 orient=0.496268]
[x=46.939002 y=14.057148 orient=0.393008]
[x=43.491410 y=12.389155 orient=2.491543]
[x=46.452010 y=16.655128 orient=2.195869]
[x=45.870849 y=12.716931 orient=0.496268]
[x=43.491410 y=12.389155 orient=2.491543]
[x=41.564902 y=11.361145 orient=5.771038]
[x=41.564902 y=11.361145 orient=5.771038]
[x=45.082584 y=9.309207 orient=2.759685]
[x=45.870849 y=12.716931 orient=0.496268]
[x=43.491410 y=12.389155 orient=2.491543]
[x=41.564902 y=11.361145 orient=5.771038]
[x=41.564902 y=11.361145 orient=5.771038]
[x=41.695354 y=16.562889 orient=2.126091]
[x=46.939002 y=14.057148 orient=0.3

[x=45.870849 y=12.716931 orient=0.496268]
[x=44.468963 y=17.342648 orient=0.340112]
[x=43.491410 y=12.389155 orient=2.491543]
[x=41.564902 y=11.361145 orient=5.771038]
[x=41.695354 y=16.562889 orient=2.126091]
[x=44.468963 y=17.342648 orient=0.340112]
[x=37.348039 y=13.952389 orient=3.678241]
[x=41.695354 y=16.562889 orient=2.126091]
[x=41.695354 y=16.562889 orient=2.126091]
[x=49.549809 y=15.705703 orient=4.758663]
[x=44.468963 y=17.342648 orient=0.340112]
[x=43.491410 y=12.389155 orient=2.491543]
[x=38.885777 y=12.888064 orient=5.045362]
[x=41.695354 y=16.562889 orient=2.126091]
[x=46.939002 y=14.057148 orient=0.393008]
[x=43.491410 y=12.389155 orient=2.491543]
[x=38.589867 y=11.880464 orient=3.575306]
[x=45.082584 y=9.309207 orient=2.759685]
[x=46.939002 y=14.057148 orient=0.393008]
[x=39.368130 y=20.379961 orient=0.068126]
[x=43.491410 y=12.389155 orient=2.491543]
[x=38.885777 y=12.888064 orient=5.045362]
[x=41.695354 y=16.562889 orient=2.126091]
[x=45.870849 y=12.716931 orient=0.4

[x=41.695354 y=16.562889 orient=2.126091]
[x=44.468963 y=17.342648 orient=0.340112]
[x=41.564902 y=11.361145 orient=5.771038]
[x=46.452010 y=16.655128 orient=2.195869]
[x=41.695354 y=16.562889 orient=2.126091]
[x=46.939002 y=14.057148 orient=0.393008]
[x=38.589867 y=11.880464 orient=3.575306]
[x=37.348039 y=13.952389 orient=3.678241]
[x=38.885777 y=12.888064 orient=5.045362]
[x=47.451792 y=14.176535 orient=0.497455]
[x=45.870849 y=12.716931 orient=0.496268]
[x=43.491410 y=12.389155 orient=2.491543]
[x=43.491410 y=12.389155 orient=2.491543]
[x=38.589867 y=11.880464 orient=3.575306]
[x=38.885777 y=12.888064 orient=5.045362]
[x=41.695354 y=16.562889 orient=2.126091]
[x=39.368130 y=20.379961 orient=0.068126]
[x=38.589867 y=11.880464 orient=3.575306]
[x=38.885777 y=12.888064 orient=5.045362]
[x=47.451792 y=14.176535 orient=0.497455]
[x=48.245957 y=10.061466 orient=4.611104]
[x=44.468963 y=17.342648 orient=0.340112]
[x=43.491410 y=12.389155 orient=2.491543]
[x=38.885777 y=12.888064 orient=5.

[x=41.564902 y=11.361145 orient=5.771038]
[x=38.885777 y=12.888064 orient=5.045362]
[x=41.695354 y=16.562889 orient=2.126091]
[x=45.870849 y=12.716931 orient=0.496268]
[x=43.491410 y=12.389155 orient=2.491543]
[x=38.589867 y=11.880464 orient=3.575306]
[x=41.564902 y=11.361145 orient=5.771038]
[x=47.451792 y=14.176535 orient=0.497455]
[x=44.660626 y=20.822293 orient=4.887577]
[x=41.695354 y=16.562889 orient=2.126091]
[x=45.870849 y=12.716931 orient=0.496268]
[x=47.715105 y=17.481047 orient=5.364998]
[x=38.589867 y=11.880464 orient=3.575306]
[x=38.885777 y=12.888064 orient=5.045362]
[x=38.885777 y=12.888064 orient=5.045362]
[x=41.695354 y=16.562889 orient=2.126091]
[x=45.870849 y=12.716931 orient=0.496268]
[x=44.468963 y=17.342648 orient=0.340112]
[x=43.491410 y=12.389155 orient=2.491543]
[x=43.491410 y=12.389155 orient=2.491543]
[x=37.348039 y=13.952389 orient=3.678241]
[x=46.452010 y=16.655128 orient=2.195869]
[x=47.451792 y=14.176535 orient=0.497455]
[x=45.870849 y=12.716931 orient=0.

[x=45.870849 y=12.716931 orient=0.496268]
[x=45.870849 y=12.716931 orient=0.496268]
[x=43.491410 y=12.389155 orient=2.491543]
[x=37.348039 y=13.952389 orient=3.678241]
[x=41.564902 y=11.361145 orient=5.771038]
[x=38.885777 y=12.888064 orient=5.045362]
[x=41.695354 y=16.562889 orient=2.126091]
[x=44.468963 y=17.342648 orient=0.340112]
[x=43.491410 y=12.389155 orient=2.491543]
[x=43.491410 y=12.389155 orient=2.491543]
[x=44.421217 y=6.954706 orient=2.539459]
[x=47.451792 y=14.176535 orient=0.497455]
[x=41.695354 y=16.562889 orient=2.126091]
[x=49.549809 y=15.705703 orient=4.758663]
[x=38.589867 y=11.880464 orient=3.575306]
[x=41.564902 y=11.361145 orient=5.771038]
[x=47.451792 y=14.176535 orient=0.497455]
[x=45.870849 y=12.716931 orient=0.496268]
[x=42.107335 y=21.622218 orient=1.859915]
[x=37.348039 y=13.952389 orient=3.678241]
